In [1]:
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm

pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', 1100)

## Предположения

- Наблюдения в логах получены независимо
- Хэдэры ботов получены перемешиванием лейблов существующих корректных юзер-агентов

### Посмотрим распределение юзер-агентов

на примере одного из лог файлов. Например 35-го по счету

In [13]:
from logParser import ParseLogsFromFolder

main_data, values_data, order_data = ParseLogsFromFolder('Logs/', 35, 36, only_order=False)

  0%|          | 0/127 [00:00<?, ?it/s]


In [99]:
df = pd.DataFrame(main_data)
# del(main_data)
print(df.shape)
distribution_frame = df.User_Agent.value_counts() / df.shape[0] 
distribution_frame.head()

(15214, 3)


Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)                                                                                                                                       0.130340
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)                                                                                                                                0.049888
Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11A465 Safari/9537.53 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)    0.035954
Mozilla/5.0 (compatible; spbot/5.0.3; +http://OpenLinkProfiler.org/bot )                                                                                                                               0.024385
Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1                                                                                                     

### На основе новой выборки получим хэдэры с намешанными лэйблами

с полученным выше распределением юзер-агентов

In [15]:
from logParser import ParseLogsFromFolder

main_data, values_data, order_data = ParseLogsFromFolder('Logs/', 36, 37, only_order=False)

  0%|          | 0/127 [00:00<?, ?it/s]


In [18]:
main = pd.DataFrame(main_data)
print(df.shape)
main.User_Agent

(16775, 3)


0        Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
1        Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_2...
2        Mozilla/5.0 (compatible; MJ12bot/v1.4.7; http:...
3        Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
4        Mozilla/5.0 (iPhone; CPU iPhone OS 10_2_1 like...
                               ...                        
15209    Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
15210    Mozilla/5.0 (compatible; YandexMetrika/2.0; +h...
15211    Mozilla/5.0 (compatible; AhrefsBot/5.2; +http:...
15212    Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Ma...
15213    Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
Name: User_Agent, dtype: object

In [24]:
np.cumsum(distribution_frame)

Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)                                                                                                                       0.127571
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)                                                                                                                0.159523
Mozilla/5.0 (compatible; MegaIndex.ru/2.0; +http://megaindex.com/crawler)                                                                                                              0.188137
Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)                                                                                                               0.216751
Mediapartners-Google                                                                                                                                                                   0.242504
                                        

In [57]:
main.shape

(15214, 3)

In [91]:
x = np.random.rand()
print(x)
print((np.cumsum(distribution_frame) > x).tail())
distribution_frame[np.cumsum(distribution_frame) > x].index[0]

0.99488
Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-G900F/G900FXXU1CPIH Build/MMB29M) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/4.0 Chrome/44.0.2403.133 Mobile Safari/537.36    False
Mozilla/5.0 (Linux; Android 6.0.1; ZB500KL Build/MMB29M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Mobile Safari/537.36                                               False
Mozilla/5.0 (Linux; Android 5.1; XT1028 Build/LPBS23.13-33.7-2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Mobile Safari/537.36                                        False
Mozilla/5.0 (Linux; Android 4.4.2; IQ4409 Quad Build/KOT49H) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.85 Mobile Safari/537.36                                           False
Mozilla/5.0 (Linux; Android 5.1.1; Redmi 3 Build/LMY47V) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.89 Mobile Safari/537.36                                                True
Name: User_Agent, dtype: bool


'Mozilla/5.0 (Linux; Android 5.1.1; Redmi 3 Build/LMY47V) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.89 Mobile Safari/537.36'

In [103]:
distribution_frame.sum()

0.998422505586937

In [106]:
sample_user_agents = []
norm_koef = distribution_frame.sum()
for sample_index in range(main.shape[0]):
    uniform_value = np.random.rand() * norm_koef 
#     print (uniform_value)
    sample_user_agents.append(
        distribution_frame[np.cumsum(distribution_frame) > uniform_value].index[0])
sample_user_agents = pd.Series(sample_user_agents)
sample_user_agents.value_counts() / main.shape[0]

Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)                                                                                                                                                            0.135204
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)                                                                                                                                                     0.048968
Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11A465 Safari/9537.53 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)                         0.038057
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36                                                                                                                0.023991
Mozilla/5.0 (compatible; spbot/5.0.3; +http://OpenLinkProfiler.org/bot )            

In [109]:
main.User_Agent = sample_user_agents
main.head()

,User_Agent,id,ip
0,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,1490562002,178.154.189.20
1,Mozilla/5.0 (iPhone; CPU iPhone OS 10_2_1 like...,1490562014,144.76.87.206
2,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,1490562020,173.234.159.250
3,Opera/9.80 (J2ME/MIDP; Opera Mini/4.1.15082/59...,1490562020,93.158.152.7
4,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,1490562033,62.183.127.125


Как видим в целом распределение юзер-агентов как мы и хотели

Обобщим в функцию и добавим в logParser

In [110]:
def GetBotSample(distr_log_index_begin, distr_log_index_end, 
                 base_log_index_begin, base_log_index_end):
    main_data, _, _ = ParseLogsFromFolder(
        'Logs/', distr_log_index_begin, distr_log_index_end, only_order=False)
    df = pd.DataFrame(main_data)
    del(main_data)
    distribution_frame = df.User_Agent.value_counts() / df.shape[0] 
    cumulative_frame = np.cumsum(distribution_frame)
    
    main_data, values_data, order_data = ParseLogsFromFolder(
        'Logs/', base_log_index_begin, base_log_index_end, only_order=False)
    main_df = pd.DataFrame(main_data)
    
    sample_user_agents = []
    norm_koef = distribution_frame.sum()
    for sample_index in range(main_df.shape[0]):
        uniform_value = np.random.rand() * norm_koef 
        sample_user_agents.append(
            distribution_frame[cumulative_frame > uniform_value].index[0])
    sample_user_agents = pd.Series(sample_user_agents)
    
    main_df.User_Agent = sample_user_agents
    
    return main_df, values_data, order_data

In [111]:
main_bot_data, values_bot_data, order_bot_data = GetBotSample(36, 37, 37, 38)
main_bot_data

  0%|          | 0/127 [00:00<?, ?it/s]


,User_Agent,id,ip
0,Mozilla/5.0 (iPhone; CPU iPhone OS 8_1 like Ma...,1493326803,37.187.134.116
1,Mozilla/4.0 (compatible; MSIE 6.0; Windows XP),1493326826,37.229.253.129
2,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,1493326826,37.229.253.129
3,Mozilla/5.0 (iPhone; CPU iPhone OS 10_2_1 like...,1493326827,37.229.253.129
4,Mozilla/5.0 (compatible; AhrefsBot/5.2; +http:...,1493326835,95.163.107.73
...,...,...,...
16174,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,1493423201,14.139.38.107
16175,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,1493423252,66.249.83.153
16176,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,1493423358,93.158.152.7
16177,Mozilla/5.0 (compatible; MJ12bot/v1.4.7; http:...,1493423368,93.158.152.7
